In [1]:
import tensorflow.compat.v1 as tf
import json
import numpy as np
import keras
import pandas as pd
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dropout, Activation


Using TensorFlow backend.


In [10]:
from google.colab import files
uploaded = files.upload()

Saving testCSV.csv to testCSV (1).csv


In [13]:
import io
import pandas as pd
training = np.genfromtxt(
    io.BytesIO(uploaded['testCSV.csv']),
    delimiter='",',
    skip_header=1,
    usecols=(0, 5),
    dtype='O',
    max_rows=10000000,
)
print(len(training))

105071


# Préparation des données

In [0]:

train_x = [x[1].decode("utf-8") for x in training]
train_y_temp = np.asarray([x[0].decode("utf-8") for x in training])

train_y = []
for y in train_y_temp:
    train_y.append(int(y[1]))

train_y = np.asarray(train_y)

max_words = 10000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_x)

dictionary = tokenizer.word_index
with open("dictionary.json", "w") as dictionary_file:
    json.dump(dictionary, dictionary_file)


def convert_text_to_index_array(text):
    # one really important thing that `text_to_word_sequence` does
    # is make all texts the same length -- in this case, the length
    # of the longest text in the set.
    return [dictionary[word] for word in kpt.text_to_word_sequence(text)]


allWordIndices = []
# for each tweet, change each token to its ID in the Tokenizer's word_index
# Changement des mots en ID de token
for text in train_x:
    wordIndices = convert_text_to_index_array(text)
    allWordIndices.append(wordIndices)

# now we have a list of all tweets converted to index arrays.
# cast as an array for future usage.
allWordIndices = np.asarray(allWordIndices)

# create one-hot matrices out of the indexed tweets
train_x = tokenizer.sequences_to_matrix(allWordIndices, mode="binary")
# treat the labels as categories
train_y = keras.utils.to_categorical(train_y, 2)


 # Modélisation et entrainement
### Première itération

In [18]:
model = Sequential(
    [
        keras.layers.Dense(512, input_shape=(max_words,), activation="relu"),
        # keras.layers.Dropout(0.5),
        # keras.layers.Embedding(10000, output_dim=64),
        # keras.layers.LSTM(128),
        keras.layers.Dense(256, activation='sigmoid'),
        #keras.layers.Dropout(0.5),
        keras.layers.Dense(2, activation="softmax")
        # keras.layers.LSTM(128),
        # keras.layers.Dense(2)
        # keras.layers.Embedding(10000, 64),
        # # keras.layers.Bidirectional(keras.layers.LSTM(64)),
        # keras.layers.LSTM(8),
        # keras.layers.Dense(64, activation='relu'),
        # keras.layers.Dense(2)
    ]
)


model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(
    train_x,
    train_y,
    batch_size=32,
    epochs=5,
    verbose=1,
    validation_split=0.1,
    shuffle=True,
)

model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')


Train on 94563 samples, validate on 10508 samples
Epoch 1/5
94563/94563 [==============================] - 15s 160us/step - loss: 0.4720 - accuracy: 0.7761 - val_loss: 0.4626 - val_accuracy: 0.7766
Epoch 2/5
94563/94563 [==============================] - 15s 159us/step - loss: 0.3341 - accuracy: 0.8560 - val_loss: 0.5489 - val_accuracy: 0.7457
Epoch 3/5
94563/94563 [==============================] - 15s 158us/step - loss: 0.1484 - accuracy: 0.9429 - val_loss: 0.8761 - val_accuracy: 0.7152
Epoch 4/5
94563/94563 [==============================] - 15s 159us/step - loss: 0.0476 - accuracy: 0.9824 - val_loss: 0.9434 - val_accuracy: 0.7833
Epoch 5/5
94563/94563 [==============================] - 15s 159us/step - loss: 0.0263 - accuracy: 0.9896 - val_loss: 1.4557 - val_accuracy: 0.7522


Surapprentissage


### 1ère Architecture du réseau
Architecture simple avec 3 couches et 770 neurones au total (entrée et sortie inclus). Nous avons mis que 5 Epochs car cela suffisait amplement pour avoir un résultats intéressant. 

In [27]:
model = Sequential(
    [
        keras.layers.Dense(512, input_shape=(max_words,), activation="relu"),
        keras.layers.Dense(256, activation='sigmoid'),
        keras.layers.Dense(2, activation="softmax")
    ]
)


model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(
    train_x,
    train_y,
    batch_size=32,
    epochs=5,
    verbose=1,
    validation_split=0.1,
    shuffle=True,
)

model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')


Train on 94563 samples, validate on 10508 samples
Epoch 1/5
94563/94563 [==============================] - 16s 167us/step - loss: 0.4680 - accuracy: 0.7784 - val_loss: 0.5783 - val_accuracy: 0.7150
Epoch 2/5
94563/94563 [==============================] - 16s 165us/step - loss: 0.3317 - accuracy: 0.8594 - val_loss: 0.5538 - val_accuracy: 0.7513
Epoch 3/5
94563/94563 [==============================] - 15s 164us/step - loss: 0.1481 - accuracy: 0.9427 - val_loss: 1.0760 - val_accuracy: 0.6746
Epoch 4/5
94563/94563 [==============================] - 15s 162us/step - loss: 0.0471 - accuracy: 0.9827 - val_loss: 1.4505 - val_accuracy: 0.7219
Epoch 5/5
94563/94563 [==============================] - 15s 163us/step - loss: 0.0268 - accuracy: 0.9899 - val_loss: 1.6356 - val_accuracy: 0.7392


### 2ème itération : 
Ajout des dropouts pour éviter le surapprentissage

In [28]:
model = Sequential(
    [
        keras.layers.Dense(512, input_shape=(max_words,), activation="relu"),
         keras.layers.Dropout(0.2),
        keras.layers.Dense(256, activation='sigmoid'),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(2, activation="softmax")
    ]
)


model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(
    train_x,
    train_y,
    batch_size=32,
    epochs=5,
    verbose=1,
    validation_split=0.1,
    shuffle=True,
)

model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')


Train on 94563 samples, validate on 10508 samples
Epoch 1/5
94563/94563 [==============================] - 17s 179us/step - loss: 0.4732 - accuracy: 0.7732 - val_loss: 0.5111 - val_accuracy: 0.7573
Epoch 2/5
94563/94563 [==============================] - 17s 176us/step - loss: 0.3530 - accuracy: 0.8445 - val_loss: 0.5249 - val_accuracy: 0.7614
Epoch 3/5
94563/94563 [==============================] - 16s 172us/step - loss: 0.2037 - accuracy: 0.9155 - val_loss: 0.8385 - val_accuracy: 0.7059
Epoch 4/5
94563/94563 [==============================] - 16s 172us/step - loss: 0.0910 - accuracy: 0.9628 - val_loss: 1.1786 - val_accuracy: 0.7383
Epoch 5/5
94563/94563 [==============================] - 16s 173us/step - loss: 0.0513 - accuracy: 0.9791 - val_loss: 1.5764 - val_accuracy: 0.7389


## 3ème itération des RNN


In [25]:
model = Sequential(
    [
        #keras.layers.Dense(512, input_shape=(max_words,), activation="relu"),
        # keras.layers.Dropout(0.5),
        keras.layers.Embedding(10000, output_dim=64),
        keras.layers.GRU(256, return_sequences=True),
        keras.layers.SimpleRNN(128),
       
        keras.layers.Dense(2)
    ]
)


model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(
    train_x,
    train_y,
    batch_size=32,
    epochs=5,
    verbose=1,
    validation_split=0.1,
    shuffle=True,
    #gradient=True
)

model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 94563 samples, validate on 10508 samples
Epoch 1/5
 1344/94563 [..............................] - ETA: 31:59:59 - loss: 7.4474 - accuracy: 0.5379

KeyboardInterrupt: ignored

### Test réel 

In [34]:
import json
import numpy as np
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
from keras.models import model_from_json


tokenizer = Tokenizer(num_words=10000)
labels = ['negative', 'positive']

# Récupération de notre dictionnaire sauvegarder
with open('dictionary.json', 'r') as dictionary_file:
    dictionary = json.load(dictionary_file)

# Vérification que tous les mots du text est dans le dictionnaire
def convert_text_to_index_array(text):
    words = kpt.text_to_word_sequence(text)
    wordIndices = []
    for word in words:
        if word in dictionary:
            wordIndices.append(dictionary[word])
        else:
            print("'%s' not in training corpus; ignoring." %(word))
    return wordIndices

# Récupération du model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
# recréation du model
model = model_from_json(loaded_model_json)
# ainsi que les poids associés durant l'entrainement
model.load_weights('model.h5')


negative sentiment; 93.509740% confidence


Test des documents

In [0]:
#evalSentence = str(input('Input a sentence to be evaluated, or Enter to quit: ')) 
evalSentence = 'you '
if len(evalSentence) != 0:
    # Formatage
    testArr = convert_text_to_index_array(evalSentence)
    input = tokenizer.sequences_to_matrix([testArr], mode='binary')
    # Prediction
    pred = model.predict(input)
    print("%s sentiment; %f%% confidence" % (labels[np.argmax(pred)], pred[0][np.argmax(pred)] * 100))
